In [55]:
import pandas as pd
import numpy as np

import arrow
import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score,\
recall_score, roc_auc_score, confusion_matrix, classification_report

warnings.filterwarnings(action='ignore')
plt.rc('font', family='Noto Sans CJK JP')
matplotlib.rc('font',family = 'Noto Sans CJK JP')

## 데이터 불러오기 

In [24]:
df_all = pd.read_csv('ALL_info.csv', engine= 'python', encoding='cp949')
df_all

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,68000000.0,0.000000,68000000.0,0.000000
1980,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1981,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1982,1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN


In [26]:
df_chaju = pd.read_csv('CreDB_chaju.csv', engine= 'python', encoding='cp949')
df_card = pd.read_csv('CreDB_card.csv', engine= 'python', encoding='cp949')
df_loan = pd.read_csv('CreDB_loan.csv', engine= 'python', encoding='cp949')
df_overdue = pd.read_csv('CreDB_overdue.csv', engine= 'python', encoding='cp949')

## 데이터 전처리 

CreDB_chaju + 'card_count' + 'card_period' + 'loan_amt' + 'loan_period' + 'over_amt' + 'over_period' + 'over_rate' 

In [27]:
df_all.drop('Unnamed: 0',axis=1,inplace =True)

In [28]:
df_all.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'income_ind_grp', 'expenditure_grp', 'child_cd', 'no_child',
       'flag_priv_biz_cd', 'card_count', 'card_period', 'loan_period',
       'Pcard_loan_amt', 'loan_amt', 'over_period', 'Pcard_over_amt',
       'over_amt', 'over_normal_amt', 'over_special_amt', 'over_s98case_amt',
       '연체율', 'loan_amt_final', '연체율특채반영', 'loan_amt_final2', '연체율S98반영'],
      dtype='object')

### chaju_id - 차주 아이디

전처리 불필요 

### credit_class - 신용 등급 

전처리 불필요 

### score_risk - 위험 점수 

전처리 불필요 

### gender - 성별 전처리 

In [29]:
# gender : 남성 = 1, 여성 = 0
df_all.loc[(df_all.gender == '남성'),'gender'] = 1
df_all.loc[(df_all.gender == '여성'),'gender'] = 0

### age - 나이

전처리 불필요 

### agegroup - 나이대 

In [30]:
#나이대 int로 변환 
df_all['agegroup']  = df_all['age']//10 * 10

### job - 직업

dummy 변수로 전처리 

In [31]:
df_all['job'].value_counts(ascending=True)

학생(재수생포함)    110
무직/기타        136
행정관리/전문직     174
전업주부         360
판매/서비스직      364
사무직          405
기능/노무직       435
Name: job, dtype: int64

### edu - 교육

dummy 변수로 전처리 

In [32]:
df_all['edu'].value_counts(ascending=True)

고등학교 재학      1
전문대학 재학      2
초등학교 졸업     13
대학원 재학      16
대학원 졸업      73
대학교 재학      84
중학교 졸업     146
전문대학 졸업    223
고등학교 졸업    707
대학교 졸업     719
Name: edu, dtype: int64

### dwell_type - 주거형태

dummy 변수로 전처리 

In [33]:
df_all['dwell_type'].value_counts(ascending=True)

영구임대        3
사택/기타       8
월세         85
전세        338
자가       1550
Name: dwell_type, dtype: int64

### house_type - 주택형태

dummy 변수로 전처리 

In [34]:
df_all['house_type'].value_counts(ascending=True)

기타              4
상가주택           11
오피스텔           19
연립/다세대/빌라     355
단독주택          405
아파트          1190
Name: house_type, dtype: int64

### marriage - 결혼상태

In [35]:
#marriage : 기혼=1, 미혼/기타=0
df_all.loc[(df_all.marriage == '기혼'),'marriage'] = 1
df_all.loc[(df_all.marriage == '미혼'),'marriage'] = 0
df_all.loc[(df_all.marriage == '기타(이혼/별거/사별'),'marriage'] = 0

### income_home_grp - 가족 평균 수입(월)

In [36]:
df_all.loc[(df_all.income_home_grp == '99만원 이하'),'income_home_grp'] = 50
df_all.loc[(df_all.income_home_grp == '100-199만원'),'income_home_grp'] = 150
df_all.loc[(df_all.income_home_grp == '200-299만원'),'income_home_grp'] = 250
df_all.loc[(df_all.income_home_grp == '300-399만원'),'income_home_grp'] = 350
df_all.loc[(df_all.income_home_grp == '400-499만원'),'income_home_grp'] = 450
df_all.loc[(df_all.income_home_grp == '500-599만원'),'income_home_grp'] = 550
df_all.loc[(df_all.income_home_grp == '600-699만원'),'income_home_grp'] = 650
df_all.loc[(df_all.income_home_grp == '700-799만원'),'income_home_grp'] = 750
df_all.loc[(df_all.income_home_grp == '800-899만원'),'income_home_grp'] = 850
df_all.loc[(df_all.income_home_grp == '900-999만원'),'income_home_grp'] = 950
df_all.loc[(df_all.income_home_grp == '1000만원 이상'),'income_home_grp'] = 2000

### income_ind_grp - 개인 평균 수입(월)

XGboost 기반 결측치 처리 

#### df_chaju 전처리

In [56]:
def change_gender(x):
    if x == '남성':
        a = 1
    else:
        a = 0
    return a

df_chaju['gender'] = df_chaju['gender'].apply(lambda x: change_gender(x))

In [57]:
def family(y):
    if y == '500-599만원':
        won = 550
    elif y == '300-399만원':
        won = 350
    elif y == '400-499만원':
        won = 450
    elif y == '600-699만원':
        won = 650
    elif y == '200-299만원':
        won = 250
    elif y == '700-799만원':
        won = 750
    elif y == '100-199만원':
        won = 150
    elif y == '800-899만원':
        won = 850
    elif y == '99만원 이하':
        won = 50
    elif y == '1000만원 이상':
        won = 1050
    elif y == '900-999만원':
        won = 950
    else:
        won = 0
        
    return won

df_chaju['income_home_grp'] = df_chaju['income_home_grp'].apply(lambda y: family(y))
df_chaju['income_ind_grp'] = df_chaju['income_ind_grp'].apply(lambda y: family(y))
df_chaju['expenditure_grp'] = df_chaju['expenditure_grp'].apply(lambda y: family(y))

In [58]:
df_chaju.drop(['chaju_id', 'agegroup','credit_class'], axis = 1, inplace = True)

In [59]:
df_raw_dummy = pd.get_dummies(df_chaju)
df_raw_dummy['income_ind_grp'] == 0
df_raw_dummy_null = df_raw_dummy[df_raw_dummy['income_ind_grp'] == 0]
df_raw_dummy_train = df_raw_dummy[df_raw_dummy['income_ind_grp'] != 0]

#### train test data생성

In [61]:
train = df_raw_dummy_train
test = df_raw_dummy_null
train.drop(['score_risk'], axis = 1, inplace = True)
df_raw_x = train.drop(["income_ind_grp"],axis = 1, inplace = False)
df_raw_y = train["income_ind_grp"]
df_train_x, df_test_x, df_train_y,df_test_y  = train_test_split(df_raw_x,df_raw_y,test_size = 0.3, random_state = 1234)

#### Xgboost 모델 생성 

In [72]:
xgb= XGBClassifier(min_samples_leaf = 60, 
                                    max_depth = 6, 
                                    n_estimators = 140,
                                    min_child_weight = .2,
                                    learning_rate= .11 )
xgb.fit(df_train_x,df_train_y)
y_pred = xgb.predict(df_test_x)
print("Score on training set:{:.3f}".format(xgb.score(df_train_x,df_train_y)))
print("Score on test set:{:.3f}".format(xgb.score(df_test_x,df_test_y)))
print("Confusion matrix:\n{}".format(confusion_matrix(df_test_y,y_pred)))
print(classification_report(df_test_y,y_pred,digits=3))

[00:57:42] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:541: 
Parameters: { min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:57:42] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score on training set:0.996
Score on test set:0.821
Confusion matrix:
[[  7   3   3   0   0   0   0   0   0   0]
 [  0  60   8   0   0   0   0   0   0   0]
 [  0   9 125   4   2   2   0   0   0   0]
 [  0   2   6  77   5   0   0   0   0   0]
 [  0   0   6   2  33   8   2   0   0   0]
 [  0   0   2   2   3  16 

#### 결측치 대체데이터 생성

In [64]:
test.drop(['score_risk'],axis=1, inplace = True)

In [65]:
final_test_x = test.drop(["income_ind_grp"],axis = 1, inplace = False)
final_test_y = test["income_ind_grp"]

In [80]:
xgb_final = XGBClassifier(min_samples_leaf = 60, 
                                    max_depth = 6, 
                                    n_estimators = 140,
                                    min_child_weight = .2,
                                    learning_rate= .11 )
xgb_final.fit(df_train_x,df_train_y)
y_pred = xgb_final.predict(final_test_x)
print(y_pred)

[01:18:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:541: 
Parameters: { min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:18:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[250 250 250 250  50 250 250 250 250  50 150 350  50 250 250 350 150 150
 350 250 250 150  50 350 250 150  50 150 250 250 150  50 150 250  50 250
 150 150 150 150 250 250  50 250 250 250  50 150  50  50 350 150  50  50
 350 150  50 250 150 350 150 250  50 250 150 250 150  50 250  50  50 250
  50  50 250 350  5

#### 결측치 대체값을 실제 데이터에 채우기

In [77]:
raw = df_all[df_all['income_ind_grp']== '모름/무응답']
raw

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,3150000.0,0.0,3150000.0,0.0
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,590000.0,0.0,590000.0,0.0
6,C-1007,C05,445,1,26,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,1170000.0,0.0,1170000.0,0.0
8,C-1009,C06,560,0,21,20,학생(재수생포함),대학교 재학,전세,연립/다세대/빌라,...,0.0,0.0,0.0,0.0,0.0,0.0,4500000.0,0.0,4500000.0,0.0
9,C-1010,C05,430,1,22,20,학생(재수생포함),대학교 재학,전세,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,C-2986,C04,368,0,46,40,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,198000000.0,0.0,198000000.0,0.0
1971,C-2988,C03,297,1,70,70,학생(재수생포함),대학교 재학,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
1974,C-2991,C06,528,0,39,30,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,23900000.0,0.0,23900000.0,0.0
1978,C-2995,C06,528,0,39,30,전업주부,전문대학 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,61340000.0,0.0,61340000.0,0.0


In [82]:
raw['income_ind_grp_1'] = [y_pred[i] for i in range(len(y_pred))]
raw

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,income_ind_grp_1
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,3150000.0,0.0,3150000.0,0.0,250
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,590000.0,0.0,590000.0,0.0,250
6,C-1007,C05,445,1,26,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,1170000.0,0.0,1170000.0,0.0,250
8,C-1009,C06,560,0,21,20,학생(재수생포함),대학교 재학,전세,연립/다세대/빌라,...,0.0,0.0,0.0,0.0,0.0,4500000.0,0.0,4500000.0,0.0,250
9,C-1010,C05,430,1,22,20,학생(재수생포함),대학교 재학,전세,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,C-2986,C04,368,0,46,40,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,198000000.0,0.0,198000000.0,0.0,250
1971,C-2988,C03,297,1,70,70,학생(재수생포함),대학교 재학,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,50
1974,C-2991,C06,528,0,39,30,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,23900000.0,0.0,23900000.0,0.0,250
1978,C-2995,C06,528,0,39,30,전업주부,전문대학 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,61340000.0,0.0,61340000.0,0.0,350


In [83]:
raw.drop('income_ind_grp', axis = 1, inplace = True)
raw.rename(columns = {'income_ind_grp_1': 'income_ind_grp'}, inplace = True)
raw

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,income_ind_grp
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,3150000.0,0.0,3150000.0,0.0,250
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,590000.0,0.0,590000.0,0.0,250
6,C-1007,C05,445,1,26,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,1170000.0,0.0,1170000.0,0.0,250
8,C-1009,C06,560,0,21,20,학생(재수생포함),대학교 재학,전세,연립/다세대/빌라,...,0.0,0.0,0.0,0.0,0.0,4500000.0,0.0,4500000.0,0.0,250
9,C-1010,C05,430,1,22,20,학생(재수생포함),대학교 재학,전세,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,C-2986,C04,368,0,46,40,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,198000000.0,0.0,198000000.0,0.0,250
1971,C-2988,C03,297,1,70,70,학생(재수생포함),대학교 재학,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,50
1974,C-2991,C06,528,0,39,30,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,23900000.0,0.0,23900000.0,0.0,250
1978,C-2995,C06,528,0,39,30,전업주부,전문대학 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,61340000.0,0.0,61340000.0,0.0,350


In [84]:
def change(y):
    if y == 550:
        won = '500-599만원'
    elif y == 350:
        won = '300-399만원'
    elif y == 450:
        won = '400-499만원'
    elif y == 650:
        won = '600-699만원'
    elif y == 250:
        won = '200-299만원'
    elif y == 750:
        won = '700-799만원'
    elif y == 150:
        won = '100-199만원'
    elif y == 850:
        won = '800-899만원'
    elif y == 50:
        won = '99만원 이하'
    elif y == 1050:
        won = '1000만원 이상'
    else:
        won = '900-999만원'
        
    return won

In [85]:
raw['income_ind_grp'] = raw['income_ind_grp'].apply(lambda y: change(y))
raw

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,income_ind_grp
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,3150000.0,0.0,3150000.0,0.0,200-299만원
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,590000.0,0.0,590000.0,0.0,200-299만원
6,C-1007,C05,445,1,26,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,1170000.0,0.0,1170000.0,0.0,200-299만원
8,C-1009,C06,560,0,21,20,학생(재수생포함),대학교 재학,전세,연립/다세대/빌라,...,0.0,0.0,0.0,0.0,0.0,4500000.0,0.0,4500000.0,0.0,200-299만원
9,C-1010,C05,430,1,22,20,학생(재수생포함),대학교 재학,전세,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,99만원 이하
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,C-2986,C04,368,0,46,40,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,198000000.0,0.0,198000000.0,0.0,200-299만원
1971,C-2988,C03,297,1,70,70,학생(재수생포함),대학교 재학,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,99만원 이하
1974,C-2991,C06,528,0,39,30,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,23900000.0,0.0,23900000.0,0.0,200-299만원
1978,C-2995,C06,528,0,39,30,전업주부,전문대학 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,61340000.0,0.0,61340000.0,0.0,300-399만원


In [87]:
original = df_all[df_all['income_ind_grp'] != '모름/무응답']
original

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,C-1001,C05,478,1,30,30,기능/노무직,대학교 졸업,자가,아파트,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
2,C-1003,C05,478,1,43,40,판매/서비스직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
4,C-1005,C05,421,1,52,50,행정관리/전문직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000
5,C-1006,C06,521,0,59,50,판매/서비스직,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,218000000.0,0.000000,218000000.0,0.000000
7,C-1008,C06,528,1,35,30,판매/서비스직,대학교 졸업,전세,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,78910000.0,0.000000,78910000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,C-2994,C05,468,1,42,40,기능/노무직,전문대학 졸업,전세,아파트,...,0.0,500000.0,500000.0,0.0,500000.0,58.139535,860000.0,58.139535,1360000.0,36.764706
1979,C-2996,C05,448,1,41,40,사무직,대학교 졸업,자가,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.000000,68000000.0,0.000000,68000000.0,0.000000
1980,C-2997,C04,308,1,37,30,사무직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1981,C-2998,C04,371,0,50,50,기능/노무직,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN


In [89]:
new_chaju_df = pd.concat([original,raw])
new_chaju_df

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,C-1001,C05,478,1,30,30,기능/노무직,대학교 졸업,자가,아파트,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
2,C-1003,C05,478,1,43,40,판매/서비스직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
4,C-1005,C05,421,1,52,50,행정관리/전문직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000
5,C-1006,C06,521,0,59,50,판매/서비스직,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,218000000.0,0.000000,218000000.0,0.000000
7,C-1008,C06,528,1,35,30,판매/서비스직,대학교 졸업,전세,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,78910000.0,0.000000,78910000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,C-2986,C04,368,0,46,40,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,198000000.0,0.000000,198000000.0,0.000000
1971,C-2988,C03,297,1,70,70,학생(재수생포함),대학교 재학,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1974,C-2991,C06,528,0,39,30,전업주부,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,23900000.0,0.000000,23900000.0,0.000000
1978,C-2995,C06,528,0,39,30,전업주부,전문대학 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,61340000.0,0.000000,61340000.0,0.000000


In [90]:
new_chaju_df = new_chaju_df.sort_index()
new_chaju_df

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,C-1001,C05,478,1,30,30,기능/노무직,대학교 졸업,자가,아파트,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000
2,C-1003,C05,478,1,43,40,판매/서비스직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000
4,C-1005,C05,421,1,52,50,행정관리/전문직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,1,41,40,사무직,대학교 졸업,자가,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.000000,68000000.0,0.000000,68000000.0,0.000000
1980,C-2997,C04,308,1,37,30,사무직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1981,C-2998,C04,371,0,50,50,기능/노무직,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1982,C-2999,C04,398,1,42,40,기능/노무직,전문대학 졸업,전세,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN


In [93]:
new_chaju_df[['income_ind_grp']]

,income_ind_grp
0,300-399만원
1,200-299만원
2,500-599만원
3,200-299만원
4,500-599만원
...,...
1979,300-399만원
1980,300-399만원
1981,100-199만원
1982,300-399만원


In [94]:
df_all[['income_ind_grp']] = new_chaju_df[['income_ind_grp']]

In [97]:
df_all[df_all['income_ind_grp'] == '모름/무응답']

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영


In [98]:
df_all

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,C-1001,C05,478,1,30,30,기능/노무직,대학교 졸업,자가,아파트,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000
2,C-1003,C05,478,1,43,40,판매/서비스직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000
4,C-1005,C05,421,1,52,50,행정관리/전문직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,1,41,40,사무직,대학교 졸업,자가,단독주택,...,0.0,0.0,0.0,0.0,0.0,0.000000,68000000.0,0.000000,68000000.0,0.000000
1980,C-2997,C04,308,1,37,30,사무직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1981,C-2998,C04,371,0,50,50,기능/노무직,고등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN
1982,C-2999,C04,398,1,42,40,기능/노무직,전문대학 졸업,전세,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN


### expenditure_grp - 개인 평균 지출(월)

In [37]:
df_all.loc[(df_all.expenditure_grp == '99만원 이하'),'expenditure_grp'] = 50
df_all.loc[(df_all.expenditure_grp == '100-199만원'),'expenditure_grp'] = 150
df_all.loc[(df_all.expenditure_grp == '200-299만원'),'expenditure_grp'] = 250
df_all.loc[(df_all.expenditure_grp == '300-399만원'),'expenditure_grp'] = 350
df_all.loc[(df_all.expenditure_grp == '400-499만원'),'expenditure_grp'] = 450
df_all.loc[(df_all.expenditure_grp == '500-599만원'),'expenditure_grp'] = 550

### child_cd - 자녀유무

In [38]:
# 자녀 있는 경우가 N, 자녀 없는 경우가 Y 로 바뀌어있음 
# child_cd : 자녀있음 = 1, 자녀없음 = 0

df_all.loc[(df_all.child_cd == 'Y'),'child_cd'] = 0
df_all.loc[(df_all.child_cd == 'N'),'child_cd'] = 1

### no_child - 자녀 수

전처리 불필요 

### flag_priv_biz_cd - 개인사업자 여부 

In [39]:
## flag_priv_biz_cd : 개인사업자 = 1, 개인사업자 아닌경우 = 0 
df_all.loc[(df_all.flag_priv_biz_cd  == 'Y'),'flag_priv_biz_cd'] = 1
df_all.loc[(df_all.flag_priv_biz_cd  == 'N'),'flag_priv_biz_cd'] = 0

### card_count - 카드 발급횟수 (파생변수)

??

### card_period_temp - 카드 기간 (파생변수) 단위:월

In [40]:
df_all.rename(columns={'card_period':'card_period_temp'},inplace=True)
df_all.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영
0,C-1001,C05,478,1,30,30,기능/노무직,대학교 졸업,자가,아파트,...,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108
1,C-1002,C05,484,0,66,60,전업주부,초등학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000
2,C-1003,C05,478,1,43,40,판매/서비스직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000
3,C-1004,C05,445,1,28,20,무직/기타,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000
4,C-1005,C05,421,1,52,50,행정관리/전문직,대학교 졸업,자가,아파트,...,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000


### card_period - 차주의 카드이용 총기간 (파생변수) 단위:월  

In [49]:
df_card_period = df_card.groupby(["chaju_id", "com_id","card_cd","card_type","card_ym"]).count()

In [50]:
df_card_period.reset_index(inplace=True)  
df_card_period.drop(["sector","check_ym","flag_priv_biz_cd"], axis = 1, inplace = True)
df_card_period.reset_index(inplace=True)  
df_card_period = df_card_period.rename(columns={'sector_cd':'period'})
df_card_period["card_ym"] = df_card_period["card_ym"].astype('str')

In [51]:
for  i in range(len(df_card_period["card_ym"])):
    date_string = df_card_period["card_ym"][i]
    date_format = '%Y%m'
    try:
        date_obj = datetime.datetime.strptime(date_string, date_format).date()
    except ValueError:
        print("Incorrect data format, should be YYYYMMDD")
    df_card_period["card_ym"][i] = date_obj   

In [52]:
start = datetime.datetime(2018, 12, 1, 0, 0, 0)
df_card_period["card_period"] = 0
for i in range(len(df_card_period["card_ym"])) :
    delta = relativedelta(start, df_card_period["card_ym"][i])  # 두 날짜의 차이 구하기
    result = 12 * delta.years + delta.months  # 두 날짜의 차이나는 개월수
    
    if result > 0 : 
        df_card_period["card_period"][i] = result + df_card_period["period"][i]
    else : 
        df_card_period["card_period"][i] = df_card_period["period"][i]

In [53]:
df_card_period[['card_period']]

,card_period
0,222
1,61
2,90
3,38
4,76
...,...
4742,36
4743,96
4744,144
4745,97


### loan_amt - 차주 대출 총금액(파생변수)

S98 포함해서 전처리

### loan_period - 대출 기간(파생변수) 단위:월

In [47]:
df_loan.drop("check_ym", axis=1).drop_duplicates()

df_loan_cnt = df_loan.groupby(["chaju_id","com_id","sector_cd","sector","loan_cd_1","loan_cd_2","loan_ym","loan_amt"]).count()

df_loan_cnt.reset_index(inplace=True)  

df_loan_cnt = df_loan_cnt.drop(["loan_prod_1","loan_prod_2"], axis = 1)

df_loan_cnt = df_loan_cnt.rename(columns={'check_ym':'add_period'})

df_loan_cnt["loan_ym"] = df_loan_cnt["loan_ym"].astype('str')

for  i in range(len(df_loan_cnt["loan_ym"])):
    date_string = df_loan_cnt["loan_ym"][i]
    date_format = '%Y%m'

    try:
        date_obj = datetime.datetime.strptime(date_string, date_format).date()
    except ValueError:
        print("Incorrect data format, should be YYYYMMDD")
    
    df_loan_cnt["loan_ym"][i] = date_obj
    
    

start = datetime.datetime(2018, 12, 1, 0, 0, 0)

df_loan_period = df_loan_cnt

df_loan_period["real_period"] = 0

for i in range(len(df_loan_period["loan_ym"])) :
    delta = relativedelta(start, df_loan_period["loan_ym"][i])  # 두 날짜의 차이 구하기
    result = 12 * delta.years + delta.months  # 두 날짜의 차이나는 개월수
    
    if result > 0 : 
        df_loan_period["real_period"][i] = result + df_loan_period["add_period"][i]
    else : 
        df_loan_period["real_period"][i] = df_loan_period["add_period"][i]
        

        
loan_period = df_loan_period["real_period"]

In [54]:
loan_period 

0        8
1       14
2        7
3        2
4        6
        ..
5360    24
5361     7
5362     7
5363    44
5364    25
Name: real_period, Length: 5365, dtype: int64

### over_amt - 차주 연체 총금액(파생변수) 

S98, 특수채권 포함해서 전처리

### over_normal_amt - 주채무자 연체 총금액(파생변수)

만들어야됨

### over_special_amt - 특수채권 연체 총금액(파생변수)

만들어야됨

### over_S98_amt - S98 연체총금액(파생변수)

만들어야됨

### over_period - 연체 기간 (파생변수) 단위 : 월

In [ ]:
df_overdue.drop("check_ym", axis=1).drop_duplicates()

df_overdue_cnt = df_overdue.groupby(["chaju_id","com_id","sector_cd","sector","over_type","over_reason_cd","over_reason","over_regist","over_ym","over_amt"]).count()

df_overdue_cnt.reset_index(inplace=True)  

df_overdue_cnt = df_overdue_cnt.rename(columns={'check_ym':'add_period'})

import datetime

df_overdue_cnt["over_ym"] = df_overdue_cnt["over_ym"].astype('str')

for  i in range(len(df_overdue_cnt["over_ym"])):
    date_string = df_overdue_cnt["over_ym"][i]
    date_format = '%Y%m'

    try:
        date_obj = datetime.datetime.strptime(date_string, date_format).date()
    except ValueError:
        print("Incorrect data format, should be YYYYMMDD")
    
    df_overdue_cnt["over_ym"][i] = date_obj
    

    
start = datetime.datetime(2018, 12, 1, 0, 0, 0)

df_overdue_period = df_overdue_cnt

df_overdue_period["real_period"] = 0

for i in range(len(df_overdue_period["over_ym"])) :
    delta = relativedelta(start, df_overdue_period["over_ym"][i])  # 두 날짜의 차이 구하기
    result = 12 * delta.years + delta.months  # 두 날짜의 차이나는 개월수
    
    if result > 0 : 
        df_overdue_period["real_period"][i] = result + df_overdue_period["add_period"][i]
    else : 
        df_overdue_period["real_period"][i] = df_overdue_period["add_period"][i]
        

over_period = df_overdue_period["real_period"]

### over_rate - 연체율 (목표변수)

In [ ]:
#over_rate로 변수명 변경
df_all.rename(columns={'연체율':'over_rate'},inplace=True)
df_all.head()

### chaju_over_rate - 개인연체율 (목표변수2)

도영이가 하고있음 

###  over_rate_weighted - 가중치를 준 연체율, 특수채권 금액을 그만큼 가중치를 줄꺼야 그러면 연체율이 달라지겠지 ,

### 가중연체액 (상환회수위험율 이런 느낌) 

### 소비소득 비율 